# General Equilibrium

## 1. Data/settings

If the local variable $t$ has been defined already, the notebook is running from another notebook. If this is not the case, a number of "local" methods have to be carried out before solving the model (e.g. setting up the relevant IO data set):

In [1]:
try:
    t = t
except NameError:
    clean_up = True
    %run StdPackages.ipynb
    name, t = 'full', 2019
    ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
    with open(f"{d['processedData']}\\glob_{t}","rb") as file: # load global settings anmed glob_name
        glob=pickle.load(file)
    db_IO = GpyDB(pickle_path = os.path.join(d['processedData'], f'{name}_{t}')) # load IO database named IO_name
    addTimeToDb(db_IO, glob.db['t'].vals, exceptions = ['sigma'])
    modules = ['_'.join([name,str(t),k]) for k in ('G','HH','Invtry','I','P','Trade','emissions')]
module = '_'.join([name, str(t)])

Load modules:

In [2]:
GE = gmsPyModels.GmsPython(name = f"{name}_GE_{t}", glob=glob, m = [loadFromPickle(n) for n in modules], s_kwargs = {'ws': ws})
robust.robust_merge_dbs(GE.s.db, db_IO, priority = 'first')

### Calibrate model

Add equilibrium module:

In [3]:
mEqui = mEquilibrium.Equilibrium(name = module, db_IO = db_IO, glob = glob)
GE.addModule(mEqui, merge_s = True)

Add a frisch elasticity to the model:

In [4]:
db_IO['frisch'] = pd.Series(0.25, index = GE.get('labor'), name = 'frisch')

Set up model, write gams code, and calibrate model to IO data:

In [5]:
GE.compile()
GE.write();
aggregateDB.subset_db(db_IO, db_IO.get('t0')) # only use t0 as target data
mCalib, cp = GE.sneakyCalib(db_IO, ws = ws, loop_kwargs = {'n': 10}, **{'cns': 'CONOPT4'})

Export solution:

In [6]:
robust.robust_merge_dbs(GE.s.db, mCalib.out_db.getTypes(['variable','scalar_variable']), priority = 'second')
GE.s.setstate('B')
with open(os.path.join(d['modules'], GE.name), "wb") as file:
    pickle.dump(GE,file)